In [1]:
import jax
import jax.numpy as jnp
import numpy as np
from pathlib import Path
import pickle, gzip, math, os, time, shutil, matplotlib as mpl, matplotlib.pyplot as plt

MNIST_URL='https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
path_data = Path('data')
path_data.mkdir(exist_ok=True)
path_gz = path_data/'mnist.pkl.gz'

In [2]:
from urllib.request import urlretrieve
if not path_gz.exists(): urlretrieve(MNIST_URL, path_gz)

with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(jnp.array, [x_train, y_train, x_valid, y_valid])

n,m = x_train.shape
c = y_train.max()+1
n,m,c

CUDA backend failed to initialize: Unable to load CUDA. Is it installed? (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


(50000, 784, Array(10, dtype=int32))

In [5]:
# num hidden
nh = 50

key = jax.random.PRNGKey(42)

w1 = jax.random.normal(key, (m,nh))
b1 = jnp.zeros(nh)
w2 = jax.random.normal(key, (nh,1))
b2 = jnp.zeros(1)

def lin(x, w, b): return x@w + b
def j_relu(x): return jnp.clip(x, a_min=0.) 

def j_model(xb):
    l1 = lin(xb, w1, b1)
    l2 = j_relu(l1)
    return lin(l2, w2, b2)


res = j_model(x_valid)
res.shape

(10000, 1)

In [6]:
def j_mse(output, targ): return jax.lax.pow((output[:,0]-targ), 2).mean()
def j_lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ jnp.transpose(w)
    w.g = (jnp.expand_dims(inp, -1) * jnp.expand_dims(out.g, 1)).sum(0)
    b.g = out.g.sum(0)
    return [inp, w, b]